# Projeto

## Data Transformation

-> Aggregation -> Scaling -> Smoothing -> Differentiation -> train-test split

1. Quarter-Hourly
2. Hourly
3. 4-Hourly

In [14]:
import pandas as pd
import numpy as np
import dslabs_functions as dslabs
from matplotlib.pyplot import figure, show, subplots, plot, legend, savefig
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from sklearn.linear_model import LinearRegression


In [8]:
### plotting UNIVARIATE Time Series
file_tag = "traffic"
target = "Total"
index="Timestamp"
### ordenar dados no csv ou nao e preciso??? last timestamp esta mal pq dados no csv nao estao ordenados por data
original_data: pd.DataFrame = pd.read_csv(
    "forecast_traffic_single.csv",
    index_col="Timestamp",
    sep=",",
    decimal=".",
    parse_dates=True,
    infer_datetime_format=True,
)

### Sort 'TimeStamp' Ascending
original_data.sort_index(inplace=True)
#original_data.to_csv("sorted_forecast_traffic.csv")
series: pd.Series = original_data[target]

print("Nr. Records = ", series.shape[0])
print("First timestamp", series.index[0])
print("Last timestamp", series.index[-1])

# Aggregations

### Group by hour
grouped_hour_means = []
for i in range(0, len(original_data), 4):
    mean_value = original_data.iloc[i:i+4]['Total'].mean()
    grouped_hour_means.append(mean_value)

ss_grouped_hour = pd.Series(grouped_hour_means, index=original_data.index[::4])

### Group by 4-hour
grouped_4_means = []
for i in range(0, len(original_data), 16):
    mean_value = original_data.iloc[i:i+16]['Total'].mean()
    grouped_4_means.append(mean_value)

ss_grouped_4 = pd.Series(grouped_4_means, index=original_data.index[::16])

Nr. Records =  2976
First timestamp 2023-10-01 00:00:00
Last timestamp 2023-10-31 23:45:00


In [10]:
def series_train_test_split(data: pd.Series, trn_pct: float = 0.90) -> tuple[pd.Series, pd.Series]:
    trn_size: int = int(len(data) * trn_pct)
    df_cp: pd.Series = data.copy()
    train: pd.Series = df_cp.iloc[:trn_size, :]
    test: pd.Series = df_cp.iloc[trn_size:]
    return train, test

## Aggregation

In [15]:
### Plot and Results for Atomic Granularity (15Min)

train_1, test_1 = series_train_test_split(original_data, trn_pct=0.90)
print(train_1)
print(test_1)
trnX_1 = np.arange(len(train_1)).reshape(-1, 1)
trnY_1 = train_1.to_numpy()
tstX_1 = np.arange(len(train_1), len(series)).reshape(-1, 1)
tstY_1 = test_1.to_numpy()
print(trnX_1)
print(trnY_1)
print(tstX_1)
print(tstY_1)

model = LinearRegression()
model.fit(trnX_1, trnY_1)

prd_trn: pd.Series = pd.Series(model.predict(trnX_1), index=train_1.index)
prd_tst: pd.Series = pd.Series(model.predict(tstX_1), index=test_1.index)

dslabs.plot_forecasting_eval(train_1, test_1, prd_trn, prd_tst, title=f"{file_tag} - Linear Regression")

                     Total
Timestamp                 
2023-10-01 00:00:00     48
2023-10-01 00:15:00     45
2023-10-01 00:30:00     39
2023-10-01 00:45:00     33
2023-10-01 01:00:00     42
...                    ...
2023-10-28 20:15:00    136
2023-10-28 20:30:00    129
2023-10-28 20:45:00    152
2023-10-28 21:00:00    150
2023-10-28 21:15:00    132

[2678 rows x 1 columns]
                     Total
Timestamp                 
2023-10-28 21:30:00    115
2023-10-28 21:45:00    138
2023-10-28 22:00:00     46
2023-10-28 22:15:00     52
2023-10-28 22:30:00     37
...                    ...
2023-10-31 22:45:00     56
2023-10-31 23:00:00     43
2023-10-31 23:15:00     59
2023-10-31 23:30:00     32
2023-10-31 23:45:00     45

[298 rows x 1 columns]
[[   0]
 [   1]
 [   2]
 ...
 [2675]
 [2676]
 [2677]]
[[ 48]
 [ 45]
 [ 39]
 ...
 [152]
 [150]
 [132]]
[[2678]
 [2679]
 [2680]
 [2681]
 [2682]
 [2683]
 [2684]
 [2685]
 [2686]
 [2687]
 [2688]
 [2689]
 [2690]
 [2691]
 [2692]
 [2693]
 [2694]
 [2695]
 [2

ValueError: Data must be 1-dimensional, got ndarray of shape (2678, 1) instead